In [1]:
from iotbx import cif
from iotbx.data_manager import DataManager
import os
os.environ["MMTBX_CCP4_MONOMER_LIB"] = "/Users/user/software/phenix/modules/chem_data/geostd"

In [2]:
dm = DataManager()
dm.process_model_file("../data/PO_data/12AS/output/12AS.updated.pdb")
dm.process_restraint_file("../data/PO_data/12AS/output/restraints/AMP.cif")
model = dm.get_model()
model.process(make_restraints=True)

In [3]:
import mmtbx
from cctbx.array_family import flex
import pandas as pd

class Selection:
    """
    A class to manage a selection on an mmtbx model manager
    
    """

    
    def __init__(self,
                 # molecule object
                 model,
                 file=None,
                 molecule_id=None,
                 
                 # selections
                 selection_string=None,
                 selection_int = None,
                 selection_bool = None,
                 
                 # other
                 array_family="flex"): # ( ["flex","np"]
        
        # initial validation
        assert isinstance(model,mmtbx.model.manager),(
            "Must provide an mmtbx model object")
        assert array_family in ["flex","np"]
        
        # set model/backend
        self.model = model
        self._file = file
        self.molecule_id = molecule_id
        self.array_family = array_family
        
        
        # set selections
        assert [selection_string,
                selection_int,
                selection_bool].count(None)==2,(
            "Must instantiate with a single type of selection (string,int,bool)")
        if type(selection_string) != type(None):
            self._selection_string = selection_string
            self._selection_bool = self.model.selection(selection_string).as_numpy_array()
            self._selection_int = np.where(self._selection_bool)[0]
            
        elif type(selection_int) != type(None):
            self._selection_int= np.array(selection_int)
            self._selection_bool = np.full(model.get_number_of_atoms(),False)
            self._selection_bool[self._selection_int] = True
        elif type(selection_bool) != type(None):
            self._selection_bool = np.array(selection_bool)
            self._selection_int = np.where(self._selection_bool)[0]
            
        # set atom identifier propertys
        self._set_atom_records(self.model)
    
        
    
    @property         
    def selection_string(self):
        if hasattr(self,"_selection_string"):
            return self._selection_string
        else:
            assert False
    
    @property
    def selection_bool(self):
        assert hasattr(self,"_selection_bool")
        if self.array_family == "flex":
            return flex.bool(self._selection_bool)
        elif self.array_family == "np":
            return self._selection_bool
            
    @property
    def selection_int(self):
        assert hasattr(self,"_selection_int")
        if self.array_family == "flex":
            return flex.int(self._selection_int)
        elif self.array_family == "np":
            return self._selection_int
    
    @property
    def atoms(self):
        atoms = self.model.get_atoms()
        return [atoms[i] for i in self.selection_int]